# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
def retrieve_titles(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    page_titles = [h3.find('a').attrs['title'] for h3 in book_container.findAll('h3')]
    return page_titles

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [3]:
import re

In [9]:
def retrieve_ratings(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    regex = re.compile("star-rating (.*)")
    star_ratings = []
    for p in book_container.findAll('p', {"class" : regex}):
        star_ratings.append(p.attrs['class'][-1])
    star_dict = {'One': 1, 'Two': 2, 'Three':3, 'Four': 4, 'Five':5} # Manually create a dictionary to translate to numeric
    star_ratings = [star_dict[s] for s in star_ratings]
    return star_ratings

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [5]:
def retrieve_prices(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    prices = [p.text for p in book_container.findAll('p', class_="price_color")]
    prices = [float(p[1:]) for p in prices]
    return prices

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [6]:
def retrieve_availabilities(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    avails = [a.text.strip() for a in book_container.findAll('p', class_="instock availability")]
    return avails

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [15]:
import pandas as pd

In [39]:
final_titles = []
star_ratings = []
prices = []
avails = []
for i in range(1,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    html_page = requests.get(url) 
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    final_titles.extend(new_titles)
    new_star_ratings = retrieve_ratings(soup)
    star_ratings.extend(new_star_ratings)
    new_prices = retrieve_prices(soup)
    prices.extend(new_prices)
    new_avails = retrieve_availabilities(soup)
    avails.extend(new_avails)
    
df = pd.DataFrame([final_titles, star_ratings, prices, avails]).transpose()
df.columns = ['Title', 'Star_Rating', 'Price_(pounds)', 'Availability']

In [40]:
df.head()

,Title,Star_Rating,Price_(pounds),Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock


In [41]:
len(df)

1000

## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [29]:
url = "http://books.toscrape.com/catalogue/page-1.html"
html_page = requests.get(url) 
soup = BeautifulSoup(html_page.content, 'html.parser')
next_button = soup.find('li', class_="next")
next_button

<li class="next"><a href="page-50.html">next</a></li>

In [30]:
next_button.find('a')

<a href="page-50.html">next</a>

In [31]:
next_button.find('a').attrs['href']

'page-50.html'

In [34]:
url = "http://books.toscrape.com/catalogue/page-50.html"
html_page = requests.get(url) 
soup = BeautifulSoup(html_page.content, 'html.parser')
next_button = soup.find('li', class_="next")
print(next_button)

None


In [43]:
#Your code here
final_titles = []
star_ratings = []
prices = []
avails = []
next_button = 'page-1.html'
current_page = 'page-1.html'
while next_button != None:    
    url = "http://books.toscrape.com/catalogue/"+current_page
    html_page = requests.get(url) 
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    final_titles.extend(new_titles)
    new_star_ratings = retrieve_ratings(soup)
    star_ratings.extend(new_star_ratings)
    new_prices = retrieve_prices(soup)
    prices.extend(new_prices)
    new_avails = retrieve_availabilities(soup)
    avails.extend(new_avails)
    next_button = soup.find('li', class_="next")
    if next_button != None:
        current_page = next_button.find('a').attrs['href']
    
df2 = pd.DataFrame([final_titles, star_ratings, prices, avails]).transpose()
df2.columns = ['Title', 'Star_Rating', 'Price_(pounds)', 'Availability']

In [44]:
df2.head()

,Title,Star_Rating,Price_(pounds),Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock


In [45]:
len(df2)

1000

In [46]:
df2.tail()

,Title,Star_Rating,Price_(pounds),Availability
995,Alice in Wonderland (Alice's Adventures in Won...,1,55.53,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock
998,1st to Die (Women's Murder Club #1),1,53.98,In stock
999,"1,000 Places to See Before You Die",5,26.08,In stock


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!